# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [9]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
!cat talks.tsv

"title"	"type"	"url_slug"	"venue"	"date"	"location"	"talk_url"	"description"
"Computer Vision and the Global Goals"	"Talk"	"cvpr-2021"	"CVPR 2021"	"Invited keynote"	"Online"	"https://www.youtube.com/watch?v=iMDawBTYQGU"	
"Satellite Analysis for Building Detection in Africa"	"Talk"	"dsa-satellite"	"Data Science Africa 2019"	"2019-06-07"	"Addis Ababa"	"https://www.youtube.com/watch?v=oazb5KyfEp4"
"Introduction to Spatial Analysis"	"Tutorial"	"dsa2019-spatial"	"Data Science Africa 2019"	"2019-06-05"	"Addis Ababa"	"https://www.youtube.com/watch?v=tCRqktM8RGM"	"[Notebook and data](https://github.com/jqug/dsa2019_spatialanalysis)"
"Machine Learning and the Humanitarian Information Gap"	"Talk"	"royalsoc-2017"	"The Royal Society"	"2017-10-30"	"London"	"http://downloads.royalsociety.org/events/2017/10/algorithms-society/Quinn.mp3"	"Meeting on [The growing ubiquity of algorithms in society: implications, impacts and innovations](https://royalsociety.org/science-events-and-lectures/2017/10/algori

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [17]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Computer Vision and the Global Goals,Talk,cvpr-2021,CVPR 2021,2021-06-20,Invited keynote,https://www.youtube.com/watch?v=iMDawBTYQGU,NaN
1,Satellite Analysis for Building Detection in A...,Talk,dsa-satellite,Data Science Africa 2019,2019-06-07,Addis Ababa,https://www.youtube.com/watch?v=oazb5KyfEp4,NaN
2,Introduction to Spatial Analysis,Tutorial,dsa2019-spatial,Data Science Africa 2019,2019-06-05,Addis Ababa,https://www.youtube.com/watch?v=tCRqktM8RGM,[Notebook and data](https://github.com/jqug/ds...
3,Machine Learning and the Humanitarian Informat...,Talk,royalsoc-2017,The Royal Society,2017-10-30,London,http://downloads.royalsociety.org/events/2017/...,Meeting on [The growing ubiquity of algorithms...
4,Interview on Talking Machines,Talk,talking-machines,The Talking Machines podcast,2017-07-01,recorded in Arusha,https://www.thetalkingmachines.com/episodes/lo...,The Long View and Machine Learning in Person (...
5,Machine Learning in the Developing World,Talk,dali2016,Data Learning and Inference (DALI) 2016,2016-04-01,Sestri Levante,https://www.youtube.com/watch?v=NNM4Z6n5_XQ,NaN
6,Overview of projects in Pulse Lab Kampala,Talk,dsa2016-plk,Data Science Africa 2016,2016-06-30,Kampala,https://www.youtube.com/watch?v=cVziXNN1VtU,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [18]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [19]:
!rm ../_talks/*.md

In [20]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\\"
    if len(str(item.talk_url)) > 3:
        md += "\n[Watch/listen](" + item.talk_url + ")\n" 
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [21]:
!ls ../_talks

2016-04-01-dali2016.md		2019-06-05-dsa2019-spatial.md
2016-06-30-dsa2016-plk.md	2019-06-07-dsa-satellite.md
2017-07-01-talking-machines.md	2021-06-20-cvpr-2021.md
2017-10-30-royalsoc-2017.md


In [22]:
!cat ../_talks/2017-07-01-talking-machines.md

---
title: "Interview on Talking Machines"
collection: talks
type: "Talk"
permalink: /talks/2017-07-01-talking-machines
venue: "The Talking Machines podcast"
date: 2017-07-01
location: "recorded in Arusha"
---

The Long View and Machine Learning in Person (season 3, episode 9), interviewed by Neil Lawrence.\
[Watch/listen](https://www.thetalkingmachines.com/episodes/long-view-and-learning-person)
